# C3D(으)로 features 뽑아내기
https://github.com/dolongbien/HumanBehaviorBKU/tree/master/C3D

In [ ]:
!nvidia-smi

Fri Sep 18 02:27:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    30W /  70W |    227MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from tensorflow.python.client import device_lib 
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14922714838118981369, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5675865712759447202
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 719772534516647753
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14640891840
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4576681906513979256
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [ ]:
# check Utubtu version
!lsb_release -a
# by default, CUDA is enabled under GPU mode of Google Colab
!ls /usr/local

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic
bin	   cuda-10.1  _gcs_config_ops.so  LICENSE.txt  setup.cfg  xgboost
cuda	   etc	      include		  man	       share
cuda-10.0  games      lib		  sbin	       src


In [ ]:
# optional
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Note
As of the CUDA 8 release, gcc 5.3 is fully supported on Ubuntu 16.06 and Fedora 23.   
As of the CUDA 9 release, gcc 6 is fully supported on Ubuntu 16.04, Ubuntu 17.04 and Fedora 25.   
The CUDA 9.2 release adds support for gcc 7

In [ ]:
# make g++, nvcc and boost work together (compatible). 
# Better install gcc before anything else
# update-alternatives --install [link] [name] [path] [priority]

!sudo apt update
!sudo apt install g++-5
!sudo update-alternatives --remove-all gcc 
!sudo update-alternatives --remove-all g++

!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 20
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 20

!sudo update-alternatives --install /usr/bin/cc cc /usr/bin/gcc 30
!sudo update-alternatives --set cc /usr/bin/gcc

!sudo update-alternatives --install /usr/bin/c++ c++ /usr/bin/g++ 30
!sudo update-alternatives --set c++ /usr/bin/g++

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [903 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [7

In [ ]:
!sudo apt-get install -f

# install dependencies

!sudo apt-get install -y libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
!sudo apt-get install libatlas-base-dev
!sudo apt-get install -y --no-install-recommends libboost-all-dev
!sudo apt-get install -y libgflags-dev libgoogle-glog-dev liblmdb-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following additional packages will be installed:
  libleveldb1v5 libprotobuf-lite10
Suggested packages:
  leveldb-doc
The following NEW packages will be installed:
  libhdf5-serial-dev libleveldb-dev libleveldb1v5 libprotobuf-dev
  libprotobuf-lite10 libsnappy-dev
0 upgraded, 6 newly installed, 0 to remove and 36 not upgraded.
Need to get 1,400 kB of archives.
After this operation, 9,964 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libhdf5-serial-dev all 1.10.0-patch1+docs-4 [2,898 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libleveldb1v5 amd64 1.20-2 [136 kB]
Get:3 http://archive.ubuntu.com

# Note
CUDA, OPENCV
- BLAS (Basic Linear Algebra Subprograms): operations like matrix
multiplication, matrix addition, both implementation for CPU(cBLAS) and
GPU(cuBLAS). provided by MKL(INTEL), ATLAS, openBLAS, etc.
- Boost: a c++ library. > Use some of its math functions and shared_pointer.
- glog,gflags provide logging & command line utilities. > Essential for
debugging.
- leveldb, lmdb: database io for your program. > Need to know this for
preparing your own data.
- protobuf: an efficient and flexible way to define data structure. > Need to
know this for defining new layer

In [ ]:
%cd /content
!git clone https://github.com/dolongbien/C3D.git

/content
Cloning into 'C3D'...
remote: Enumerating objects: 1737, done.
remote: Total 1737 (delta 0), reused 0 (delta 0), pack-reused 1737
Receiving objects: 100% (1737/1737), 28.62 MiB | 28.65 MiB/s, done.
Resolving deltas: 100% (597/597), done.


In [ ]:
# compile

%cd /content/C3D/C3D-v1.0/
!make clean
!make all -j8

/content/C3D/C3D-v1.0
protoc --proto_path=src --cpp_out=build/src src/caffe/proto/caffe.proto
protoc --proto_path=src --cpp_out=build/src src/caffe/proto/caffe_pretty_print.proto
/usr/local/cuda/bin/nvcc -ccbin=/usr/bin/g++ -Xcompiler -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode=arch=compute_50,code=sm_50 -gencode=arch=compute_75,code=sm_75  -c src/caffe/util/im2col.cu -o build/src/caffe/util/im2col.cuo
/usr/local/cuda/bin/nvcc -ccbin=/usr/bin/g++ -Xcompiler -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=s

In [ ]:
!make test

/usr/bin/g++ src/caffe/test/test_power_layer.cpp -pthread -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -c -o build/src/caffe/test/test_power_layer.o

/usr/bin/g++ src/caffe/test/test_util_blas.cpp -pthread -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -c -o build/src/caffe/test/test_util_blas.o

/usr/bin/g++ src/caffe/test/test_filler.cpp -pthread -fPIC -DNDEBUG -O2 -I/usr/local/include/python2.7 -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I/usr/local/include -I/usr/include/hdf5/serial/ -Ibuild/src -I./src -I./include -I/usr/local/cuda/include -c -o build/src/caffe/test/test_filler.o

/usr/bin/g++ src/caffe/test/test_euclidean_loss_

In [ ]:
# 이건 오래걸리기도 하고 옵션으로 하는거라서 시간상 건너뛰자
# optional
# !make runtest

In [ ]:
%cd /content/C3D/C3D-v1.0/examples/c3d_feature_extraction/
!wget -O conv3d_deepnetA_sport1m_iter_1900000 https://www.dropbox.com/s/vr8ckp0pxgbldhs/conv3d_deepnetA_sport1m_iter_1900000?dl=0

/content/C3D/C3D-v1.0/examples/c3d_feature_extraction
--2020-09-18 00:15:09--  https://www.dropbox.com/s/vr8ckp0pxgbldhs/conv3d_deepnetA_sport1m_iter_1900000?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/vr8ckp0pxgbldhs/conv3d_deepnetA_sport1m_iter_1900000 [following]
--2020-09-18 00:15:09--  https://www.dropbox.com/s/raw/vr8ckp0pxgbldhs/conv3d_deepnetA_sport1m_iter_1900000
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3b23495a7024d8bf926ab90fac.dl.dropboxusercontent.com/cd/0/inline/A_k5dbMV2HTqpELXfzg4s_g8hqLxweYN8pkQZN92SkVayrUElldNjJLTqH3ds3Muxz9JqzIaO8plPlUbkNKkyKwmkd48TNWgyV28RIfXisyV0A/file# [following]
--2020-09-18 00:15:10--  https://uc3b23495a7024d8bf926ab90fac.dl.dropboxusercontent.com/cd/0/inline/A_k5

# Extract feature, check output folder in c3d_sport1m_feature_extraction_video.sh

In [ ]:
# extract feature
%cd /content/C3D/C3D-v1.0/examples/c3d_feature_extraction
!sudo bash c3d_sport1m_feature_extraction_video.sh

/content/C3D/C3D-v1.0/examples/c3d_feature_extraction
E0918 04:46:11.562969 10086 extract_image_features.cpp:64] Using GPU #0
I0918 04:46:11.564587 10086 net.cpp:76] Creating Layer data
I0918 04:46:11.564617 10086 net.cpp:112] data -> data
I0918 04:46:11.564693 10086 net.cpp:112] data -> label
I0918 04:46:11.564749 10086 video_data_layer.cpp:283] Opening file prototxt/input_list_video.txt
I0918 04:46:11.565218 10086 video_data_layer.cpp:317] A total of 831 video chunks.
I0918 04:46:12.958674 10086 video_data_layer.cpp:365] output data size: 50,3,16,112,112
I0918 04:46:12.958710 10086 video_data_layer.cpp:387] Loading mean file from sport1m_train16_128_mean.binaryproto
I0918 04:46:12.972419 10086 net.cpp:127] Top shape: 50 3 16 112 112 (30105600)
I0918 04:46:12.972445 10086 net.cpp:127] Top shape: 50 1 1 1 1 (50)
I0918 04:46:12.972451 10086 net.cpp:159] data does not need backward computation.
I0918 04:46:12.972466 10086 net.cpp:76] Creating Layer conv1a
I0918 04:46:12.972476 10086 net.

# generate_inout_file.py

In [ ]:
# %cd /content/gdrive/My\ Drive/C3D/C3D-v1.0/examples/c3d_feature_extraction/input/Traning_Normal_Part1_Video
%cd /content/C3D/C3D-v1.0/examples/c3d_feature_extraction/input/Traning_Normal_Part1_Video
import os, os.path
import cv2
import glob
import math

f_input = open('input_list_video.txt', 'w')
f_output = open('output_list_video_prefix.txt','w')
sh_out = open('c3d_sport1m_feature_extraction_video.sh', 'w')

interval_v = 16

cmd1 = 'time GLOG_logtosterr=1 ../../build/tools/extract_image_features.bin prototxt/c3d_sport1m_feature_extractor_video.prototxt conv3d_deepnetA_sport1m_iter_1900000'

cmd2 = 'prototxt/output_list_video_prefix.txt fc6-1'

gpu_id = 0
batch_size = 50
batch_num = 1
max_batch_num = 1

# read file path name in alphabet order
file_list_v = sorted(glob.glob("*.mp4"))

# each video, generate input/output file
for index in range(len(file_list_v)):
	
	# file path of video
	loc = file_list_v[index]
	cap=cv2.VideoCapture(loc)
	
	# generate .sh file
	sh_out.write('rm -rf output/Traning_Normal_Part1_FC/' + loc[:-4])
	sh_out.write('\n')
	sh_out.write('mkdir -p output/Traning_Normal_Part1_FC/' + loc[:-4])
	sh_out.write('\n')
	
	# number of frames
	total_frames = int(cap.get(7))
	print(index+1, total_frames, sep='\tframes=')
	
	# calculate batch number
	curr_batch_num = math.ceil(total_frames/batch_size)
	max_batch_num = batch_num
	if curr_batch_num >= max_batch_num:
		batch_num = curr_batch_num
	else:
		batch_num = max_batch_num
	

	counter = 0   

	# generate file
	while (counter <= total_frames-interval_v):
		f_input.write('input/Traning_Normal_Part1_Video/' + loc + ' ' + str(counter) + ' ' + str(0))
		f_input.write('\n')
		f_output.write('output/Traning_Normal_Part1_FC/' + loc[:-4] + '/' + str(counter).zfill(6) )
		f_output.write('\n')
		counter = counter + interval_v

# write command to bash file
parameter = ' ' + str(gpu_id) + ' ' + str(batch_size) + ' ' + str(batch_num) + ' ' 
sh_out.write(cmd1 + parameter + cmd2)

print('-------- PARAMETER --------')
print('GPU ID is', gpu_id)
print('Batch size is', batch_size)
print('Number of batch is', batch_num)

f_input.close()
f_output.close()
sh_out.close()

/content/C3D/C3D-v1.0/examples/c3d_feature_extraction/input/Traning_Normal_Part1_Video
1	frames=1415
2	frames=1034
3	frames=1457
4	frames=1822
5	frames=1676
6	frames=969
7	frames=585
8	frames=1220
9	frames=2344
10	frames=849
-------- PARAMETER --------
GPU ID is 0
Batch size is 50
Number of batch is 47
